In [179]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____


# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [93]:
# import PyPDF2
# import re
# import win32com.client
# from win32com.client.dynamic import Dispatch
import os
from pdf2image import convert_from_path
# import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
# import numpy as np
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [207]:

# FUnction to extract words from a pdf and save it in as a text file

def pdf_2_txt(path):          
    for file in os.listdir(path): # goes through the folders'
        if file.endswith('.pdf'): # finds pdfs
            p = os.path.join(path,file) # grabs path of pdf
            pdfpages = convert_from_path(p, 500) # Converting each page in the pdf into image file with 500 dpi
            image_counter = 1
            for page in pdfpages:
                filename = f"page_{image_counter}.jpg"
                # Save the image of the page in system
                page.save(filename, 'JPEG')
                image_counter+=1
                text = str(((pytesseract.image_to_string(Image.open(filename))))) # takes image and process into text
                # delete image
                os.remove(filename)
                text = text.replace('-\n', '') # some text processing
                                
                # Checking if the page contains the "Course objective or 
                # description or outline or goals or Learning Outcomes followed by an "end of string" and
                # a new line (As a title)"
                
                if re.match(re.compile("^Course\s(Outline|Objectives|Objective|Description|Goals)$\n"),
                            text) or re.match(re.compile("^Learing Outcomes$\n"), text):
                    filename = filename.replace('.jpg','') # Naming
                    with open(f'{filename}.txt', 'w+') as f: # Opens a text file and writes the characters onto
                        print(text, file=f)
                        print(f'Page_{image_counter}.txt has been saved into {os.getcwd()}')
                    break # stops iteration as soon as page is found               
                
    return #text
                    
                
        


In [205]:

# path = os.path.join(os.getcwd(),'syllabus.pdf')
path = os.getcwd()
pdf_2_txt(path)
# pages = convert_from_path(path, 500) 
# for page in pages:
#     page.save('test.jpg')

Page_3.txt has been saved into C:\Users\15713\Desktop


In [49]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib

# % matplotlib inline

In [51]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords 

In [176]:
def combine_pages(path):
    # Combining pages
    pages = ''
    for file in os.listdir(path):
        if file.endswith('.txt'):
            with codecs.open(file, 'r') as f: 
                page = f.read()
                pages+=page
    return pages

# pages = combine_pages(path)
# pages

In [144]:
# text = pages
# tokens_P = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
# tokens = WordPunctTokenizer().tokenize(text)


In [180]:
# getting a list of unecessary words like 'the', 'or',... to avoid
avoid = stopwords.words('english')
avoid.append('edu') # adding our own word to the list

In [255]:
# Function to tokenize a text file into sentences or phrases
def get_sentence_counter(path):
    path = os.getcwd()
    for file in os.listdir():
        if file.endswith('.txt'):
            with codecs.open(file, 'r') as f: 
                text = f.read()
                # groups all words and white-spaces, then finds words until a punctuation is reached
                regex_of_sentence = re.findall('([\w\s]{0,})[^\w\s]', text) 
                avoid = re.findall('\d', text)
                avoid.append('')
    regex_of_sentence = [x for x in regex_of_sentence if x not in avoid]
    # for i in regex_of_sentence:
    #     print(i)
    return collections.Counter(regex_of_sentence), len(regex_of_sentence)


In [204]:
# Function to tokenize a text file into words
def get_text_counter(text, avoid, word_pattern):
#     tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = WordPunctTokenizer().tokenize(text)
    tokens = list(map(lambda x: x.lower(), tokens)) # converting everything to lower-case
    # cleaning up tokens by getting rid of stopwords and finding word characters
    Puncs = [token for token in tokens if re.match(re.compile("\W"), token)] # finding all punctuations
    Nums = [token for token in tokens if re.match(re.compile("\d"), token)] # finding all digit characters
    # adding punctuations and digits to the list of avoided characters/words
    avoid = Puncs + Nums
    tokens = [token for token in tokens if re.match(word_pattern, token) and token not in avoid] 
    # collections.counter() returns the tokens and their frequency as a dictionary ({freq: token})
    return collections.Counter(tokens), len(tokens) 

In [259]:
# Function to make dataframe for both tokenized words and sentences with their abs and relative freqz
def make_df(counter, size, ind_name):
    abs_freq = np.array([x[1] for x in counter]) # the absolute freqz is found from the collections.counter()
    rel_freq = abs_freq/size # relative freqz is calculated by dividing abs_freqz by the size
    index = [x[0] for x in counter] # the tokens are used as an index 
    # creating the dataframe
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, 
                      index=index,
                      columns=["Absolute frequency","Relative frequency"])
    # Naming the index title
    df.index.name = ind_name
    return df

In [258]:
# Implementing functions for sentences or phrases
path = os.getcwd()
sentence_count, size = get_sentence_counter(path)
df_sentence = make_df(sentence_count.most_common()[:-n-1:-1], size,
             ind_name='Least Commonly\n Occuring Sentences') # finds the least commonly occuring sentences
df_sentence.head(50)

,Absolute frequency,Relative frequency
Least Common Words,,
Your grade will be determined by your participation in discussions and submitted\nreviews,1.0,0.011494
You will be a reviewer or reader for 4 grants and participate in a simulated NIH\nstudy section,1.0,0.011494
\n\nAssignment,1.0,0.011494
NSF and NIH pdfs and web sites,1.0,0.011494
Writing and Reviewing Research Proposals\nReading,1.0,0.011494
\n\n6,1.0,0.011494
quiz on Leedy Ch 11,1.0,0.011494
Leedy Ch11 and completion of the Linked In course on R\n\nEvaluation,1.0,0.011494
Quantitative Analyses and Review of Statistical Procedures\nReading,1.0,0.011494


In [260]:
# Implementing functions for sentences or phrases
path = os.getcwd()
pages = combine_pages(path) # combining every text file in folder
word_count, size = get_text_counter(pages, avoid, word_pattern) # returns tokens freqz and count of tokens
n = 1000 # 1000 of least commonly occuring words
# least commonly occuring words are chosen with the assumption that a syllabus will only mention
# technical terms once or twice while other words like 'students' and so on could be redundant
df_word = make_df(word_count.most_common()[:-n-1:-1], size,
             ind_name='Least Commonly\n Occuring Words') # finds the least commonly occuring words
df_word.head(50)

In [229]:
# Just showing a complete list of the words tokenized
n = 1000
print([x[0] for x in word_count.most_common()[:-n-1:-1]])

['proposal', 'page', 'submit', 'reviews', 'submitted', 'discussions', 'participation', 'determined', 'grade', 'section', 'study', 'simulated', 'participate', 'grants', 'reader', 'or', 'reviewer', 'sites', 'reviewing', 'r', 'linked', 'completion', 'ch11', 'procedures', 'statistical', 'analyses', 'perk', 'editors', 'com', 'www', 'https', 'site', 'variables', 'confounding', 'bias', 'sources', 'sampling', 'design', 'experimental', 'qualitative', 'weaknesses', 'strengths', 'list', 'suggestions', 's', 'not', 'does', 'it', 'corresponds', 'identify', 'according', 'one', 'presentation', 'borja', 'bates', 'manuscript', 'weeks', 'due', 'ms', 'phd', 'references', 'pages', 'report', 'ch3', 'skim', 'magee', 'chris', 'zotero', 'software', 'manager', 'reference', 'calcagno', 'theresa', 'searches', 'databases', 'library', 'do', 'how', 'chapters', 'articles', 'present', 'supervisor', 'with', 'thesis', 'ormrod', 'introduction', 'red', 'are', 'assignments', 'evaluations', 'content', 'presentations', 'post

In [262]:
# Just showing a complete list of the words tokenized
n = 1000
print([x[0] for x in sentence_count.most_common()[:-n-1:-1]])

[' Your grade will be determined by your participation in discussions and submitted\nreviews', ' You will be a reviewer or reader for 4 grants and participate in a simulated NIH\nstudy section', '\n\nAssignment', ' NSF and NIH pdfs and web sites', ' Writing and Reviewing Research Proposals\nReading ', '\n\n6', ' quiz on Leedy Ch 11', ' Leedy Ch11 and completion of the Linked In course on R\n\nEvaluation', ' Quantitative Analyses and Review of Statistical Procedures\nReading ', '7\n\n5', ' quiz on Leedy Ch 4', 'perk\nEvaluation', 'editors', 'com', 'elsevier', 'www', ' Publication Ethics from Elsevier web site\n\nhttps', ' Sections of Leedy Ch 4', '\nReading ', ' Research Ethics', ' Confounding Variables', ' Sources of Bias', '\nSampling', ' Experimental Design', ' Quantitative and Qualitative Research', ' The Research Process ', '\n\n4', ' List and discuss Strengths and Weaknesses of the Writing', 's suggestions', ' according to the Baylor Sections and identify where the paper correspon